深度模型有关数值稳定性的典型问题是衰减（vanishing）和爆炸（explosion）。

## 梯度消失和梯度爆炸
   当神经网络的层数较多时，模型的数值稳定性容易变差。假设一个层数为 $L$ 的多层感知机的第$l$层$\boldsymbol{H}^{(l)}$的权重参数为$\boldsymbol{W}^{(l)}$，输出层$\boldsymbol{H}^{(L)}$的权重参数为$\boldsymbol{W}^{(L)}$。为了便于讨论，不考虑偏差参数，且设所有隐藏层的激活函数为恒等映射（identity mapping）$\phi(x) = x$。给定输入$\boldsymbol{X}$，多层感知机的第$l$层的输出$\boldsymbol{H}^{(l)} = \boldsymbol{X} \boldsymbol{W}^{(1)} \boldsymbol{W}^{(2)} \ldots \boldsymbol{W}^{(l)}$。此时，如果层数$l$较大，$\boldsymbol{H}^{(l)}$的计算可能会出现衰减或爆炸。举个例子，假设输入和所有层的权重参数都是标量，如权重参数为0.2和5，多层感知机的第30层输出为输入$\boldsymbol{X}$分别与$0.2^{30} \approx 1 \times 10^{-21}$（衰减）和$5^{30} \approx 9 \times 10^{20}$（爆炸）的乘积。类似地，当层数较多时，梯度的计算也更容易出现衰减或爆炸。

## 随机初始化模型参数
在神经网络中，通常需要随机初始化模型参数。假设感知机的输出层只保留一个输出单元$o_1$，且隐藏层使用相同的激活函数。如果将每个隐藏单元的参数都初始化为相等的值，那么在正向传播时每个隐藏单元将根据相同的输入计算出相同的值，并传递至输出层。在反向传播中，每个隐藏单元的参数梯度值相等。因此，这些参数在使用基于梯度的优化算法迭代后值依然相等。之后的迭代也是如此。在这种情况下，无论隐藏单元有多少，隐藏层本质上只有1个隐藏单元在发挥作用。因此，正如在前面的实验中所做的那样，我们通常将神经网络的模型参数，特别是权重参数，进行随机初始化。

###  PyTorch的默认随机初始化
随机初始化模型参数的方法有很多。我们使用torch.nn.init.normal_()使模型net的权重参数采用正态分布的随机初始化方式。不过，PyTorch中nn.Module的模块参数都采取了较为合理的初始化策略（不同类型的layer具体采样的哪一种初始化方法的可参考源代码），因此一般不用我们考虑。

### Xavier随机初始化
还有一种比较常用的随机初始化方法叫作Xavier随机初始化。 假设某全连接层的输入个数为$a$，输出个数为$b$，Xavier随机初始化将使该层中权重参数的每个元素都随机采样于均匀分布

$$U\left(-\sqrt{\frac{6}{a+b}}, \sqrt{\frac{6}{a+b}}\right).$$

它的设计主要考虑到，模型参数初始化后，每层输出的方差不该受该层输入个数影响，且每层梯度的方差也不该受该层输出个数影响。

## 下列环境因素对模型训练有影响

### 协变量偏移
训练时模型的输入与测试时模型的输入不同，特征分布发生了变化（即协变量的变化），即P（x）改变了，但P（y∣x）保持不变。
如果一个在冬季部署的物品推荐系统在夏季的物品推荐列表中出现了圣诞礼物，可以推断该系统没有考虑到协变量偏移，这是因为在夏季的物品推荐系统与冬季相比，时间或者说季节发生了变化，导致了夏季推荐圣诞礼物的不合理的现象。

### 标签偏移
训练数据集，数据很少只包含流感p(y)的样本。而测试数据集有流感p(y)和流感q(y)，其中不变的是流感症状p(x|y)，这样就会发生标签偏移。
训练时模型的标签与测试时模型的输出不同，即在p(y/x)未变，p(y)却变了

如果数据量足够的情况下，确保训练数据集和测试集中的数据取自同一个数据集，可以防止协变量偏移和标签偏移。如果数据量很少，少到测试集中存在训练集中未包含的标签，就会发生标签偏移。


### 概念偏移
基于所学习的概念应时间和空间的不同产生变化，即y与x本身的实际意义发生了变化